# SSU
## This script loads, normalize and filter abundance data resulting from SSU reference library 

In [18]:
import pandas as pd
pd.options.mode.chained_assignment = None

## Load the abundace and trim data 

In [19]:
abund_tb=pd.read_csv("../data/batch1_20230720/SSU/BFTMS23071402 (single)OTU (Table).csv")
trim_tbl=pd.read_csv("../data/batch1_20230720/Trim_summary.csv")

## Rename the columns by removing the keyword abundance and special characters:

In [20]:
# Rename the columns by removing the keyword abundance and BFTMS
abund_tb.columns = [col.replace("Abundance", '') for col in abund_tb.columns]
abund_tb.columns = [col.replace(" ", '') for col in abund_tb.columns]
#abund_tb.columns = [col.replace("BFTMS", '') for col in abund_tb.columns]

##add a control column that gets values from the spcified control sample
control="BFTMS23071417"

## alternatively can be read from the sample key

#abund_tb["control"]=abund_tb[control]

samples=trim_tbl["Name"].astype(str).tolist()
#samples.append('control')

abund_tb['reference_lib']='SSU'

## mark species with high risk of false positive detection in a column "false_pos_prone"
### for ssu it includes 'Acinetobacter baumannii', 'Klebsiella oxytoca', 'Enterobacter cloacae','Citrobacter freundii'

In [22]:

# first split the ID column into species group name and species ID
abund_tb[['species group', 'species id']] = abund_tb['ID'].str.split(';', expand=True)
## create a new column "false_pos_prone" and mark species with high risk of false positive detection
abund_tb['false_pos_prone']='No'


def mark_species(df, ID_column, false_pos_tag, species_list):
    # Update Column B to True for rows where values in Column A start with any of the strings in the search list
    for name in species_list:
        df.loc[df[ID_column].str.startswith(name), false_pos_tag] = 'Yes'
    return df

false_pos_prone_spicies=['Acinetobacter baumannii', 'Klebsiella oxytoca', 'Enterobacter cloacae',
                         'Citrobacter freundii']
abund_tb = mark_species(abund_tb, 'species group', 'false_pos_prone', false_pos_prone_spicies)
#abund_tb.loc[abund_tb['species group'].isin(false_pos_prone_spicies), 'false_pos_prone'] = 'Yes'

# Check the structure and integrity of the abundance table

In [23]:
abund_tb.head(5)

,ID,Name,Taxonomy,Combined,Min,Max,Mean,Median,Std,BFTMS23071402,...,BFTMS23071413,BFTMS23071414,BFTMS23071415,BFTMS23071416,BFTMS23071417,Sequence,reference_lib,species group,species id,false_pos_prone
0,Pseudomonas amygdali pv. morsprunorum; AB00144...,AB001445.1.1538,Bacteria; Proteobacteria; Gammaproteobacteria;...,1770,0,276,110.6250,118.0,79.300168,181,...,163,41,8,37,164,CCTGGGCTACACACGTGCTACAATGGTCGGTACAGAGGGTTGCCAA...,SSU,Pseudomonas amygdali pv. morsprunorum,AB001445.1.1538,No
1,Dickeya phage phiDP10.3; KM209255.204.1909,KM209255.204.1909,Bacteria; Proteobacteria; Gammaproteobacteria;...,3853,3,545,240.8125,223.5,168.319624,351,...,514,96,25,54,346,TACACACGTGCTACAATGGCGCATACAAAGAGAAGCGACCTCGCGA...,SSU,Dickeya phage phiDP10.3,KM209255.204.1909,No
2,Streptococcus porcinus; AB002523.1.1496,AB002523.1.1496,Bacteria; Firmicutes; Bacilli; Lactobacillales...,2,0,2,0.1250,0.0,0.500000,0,...,0,0,0,0,0,CAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGAGT...,SSU,Streptococcus porcinus,AB002523.1.1496,No
3,Pseudomonas sp. NR25; JN082749.1.1447,JN082749.1.1447,Bacteria; Proteobacteria; Gammaproteobacteria;...,32,0,11,2.0000,0.5,3.224903,0,...,3,0,0,2,4,AATGCCTAGGAATCTGCCTGGTAGTGGGGGACAACGTTTCGAAAGG...,SSU,Pseudomonas sp. NR25,JN082749.1.1447,No
4,Bradyrhizobium sp. ORS 3635; JN085489.1.1385,JN085489.1.1385,Bacteria; Proteobacteria; Alphaproteobacteria;...,62,0,43,3.8750,0.0,10.855874,11,...,0,1,0,0,6,TCAAGTCCTCATGGCCCTTACGGGCTGGGCTACACACGTGCTACAA...,SSU,Bradyrhizobium sp. ORS 3635,JN085489.1.1385,No


## data normalization & corection for control $ S:N

In [24]:
## data normalization and corection for control

resid_val=0.00 # this value will be added to sample and control to avoid deviding to 0

# normalize all abundances based on the total reads for each sample
for samp in samples:
    ## read the trimmed reads from trim_tbl and get the value based on the sample name
    trimmed_reads=trim_tbl[trim_tbl["Name"]==samp]["Trimmed sequences"].values[0]
    abund_tb[samp+"_Normalized"]=abund_tb[samp]*trimmed_reads/1000000

## correct normized abundance by deducing control abundance
for samp in samples:
    abund_tb[samp+"_Normalized_corrected"]=abund_tb[samp+"_Normalized"]-abund_tb[control+"_Normalized"]
    ## calculate signal to noize 
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

## ## calculate signal to noise
for samp in samples:
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

## pick one sample by name and review the accuracy of the normalization and background corrections

In [25]:
#check one sample and compare it against the results from manual analysis
sample='BFTMS23071405'
sample_df=abund_tb[['ID', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']]
sample_df.sort_values(by=sample+'_Normalized_corrected', ascending=False)
sample_df.head(5)

,ID,BFTMS23071405,BFTMS23071417,BFTMS23071405_Normalized,BFTMS23071417_Normalized,BFTMS23071405_Normalized_corrected,BFTMS23071405_S:N
0,Pseudomonas amygdali pv. morsprunorum; AB00144...,73,164,12.643673,16.515292,-3.871619,-0.234426
1,Dickeya phage phiDP10.3; KM209255.204.1909,194,346,33.600994,34.843238,-1.242244,-0.035652
2,Streptococcus porcinus; AB002523.1.1496,0,0,0.000000,0.000000,0.000000,NaN
3,Pseudomonas sp. NR25; JN082749.1.1447,1,4,0.173201,0.402812,-0.229611,-0.570020
4,Bradyrhizobium sp. ORS 3635; JN085489.1.1385,0,6,0.000000,0.604218,-0.604218,-1.000000


# Apply the filtering criteria for SSU

In [29]:
## ssu strict

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]

    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=1000) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=10)) &
                            (sample_abund['normalized_corrected']>=5)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=25) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=10)) &
                            (sample_abund['normalized_corrected']>=5))
                            ]
    
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='SSU_strict'
select_IDs.to_csv('../output_tables/batch1_20230720/SSU_strict.csv')


In [27]:
## ssu moderate

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]


    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=1000) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=5)) &
                            (sample_abund['normalized_corrected']>=3)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=7) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=5)) &
                            (sample_abund['normalized_corrected']>=3))
                            ]
    
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='SSU_moderate'
select_IDs.to_csv('../output_tables/batch1_20230720/SSU_moderate.csv')




In [28]:
## ssu loose

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]


    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=1000) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=3)) &
                            (sample_abund['normalized_corrected']>=3)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=5) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=3)) &
                            (sample_abund['normalized_corrected']>=3))
                            ]
    
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='SSU_loose'
select_IDs.to_csv('../output_tables/batch1_20230720/SSU_loose.csv')


